# Chapter 12: Physically Based Rendering (PBR)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_12_physically_based_rendering.ipynb)

**Physically Based Rendering (PBR)** is a shading model based on real-world physics that ensures energy conservation and produces realistic materials across all lighting conditions. PBR has become the industry standard in games, films, and real-time graphics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
from typing import Tuple, List, Optional
from dataclasses import dataclass

In [ ]:
class Vec3:
    """3D Vector class"""
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x = x
        self.y = y
        self.z = z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def __neg__(self):
        return Vec3(-self.x, -self.y, -self.z)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return self / l
        return Vec3(0, 0, 0)
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

class Color:
    """RGB Color class"""
    def __init__(self, r: float = 0.0, g: float = 0.0, b: float = 0.0):
        self.r = r
        self.g = g
        self.b = b
    
    def __add__(self, other):
        return Color(self.r + other.r, self.g + other.g, self.b + other.b)
    
    def __mul__(self, other):
        if isinstance(other, Color):
            return Color(self.r * other.r, self.g * other.g, self.b * other.b)
        else:
            return Color(self.r * other, self.g * other, self.b * other)
    
    def __truediv__(self, scalar: float):
        return Color(self.r / scalar, self.g / scalar, self.b / scalar)
    
    def clamp(self, min_val: float = 0.0, max_val: float = 1.0):
        return Color(
            max(min_val, min(max_val, self.r)),
            max(min_val, min(max_val, self.g)),
            max(min_val, min(max_val, self.b))
        )
    
    def to_tuple(self) -> Tuple[float, float, float]:
        return (self.r, self.g, self.b)
    
    def to_vec3(self) -> Vec3:
        return Vec3(self.r, self.g, self.b)

print("✓ Base classes loaded")

## 1. PBR Principles

### Energy Conservation

**Energy conservation** states that reflected light can never exceed incident light:

$$
\int_{\Omega} f_r(\mathbf{p}, \omega_i, \omega_o) \, (\mathbf{n} \cdot \omega_i) \, d\omega_i \leq 1
$$

This ensures physically plausible materials.

### Metallic Workflow

PBR typically uses the **metallic workflow** with parameters:

- **Base Color (Albedo)**: $\mathbf{c}$ - Diffuse color for dielectrics, reflectance for metals
- **Metallic**: $m \in [0, 1]$ - 0 = dielectric, 1 = metal
- **Roughness**: $r \in [0, 1]$ - 0 = smooth, 1 = rough
- **Normal**: $\mathbf{n}$ - Surface normal (from normal maps)
- **Ambient Occlusion**: $ao \in [0, 1]$ - Cavity shading

### Physical Correctness

1. **Energy conservation**: Total reflected ≤ incident
2. **Reciprocity**: $f_r(\omega_i, \omega_o) = f_r(\omega_o, \omega_i)$
3. **Fresnel effect**: Reflectance increases at grazing angles
4. **Linear color space**: Calculations in linear RGB, not sRGB

## 2. Microfacet Theory

**Microfacet BRDF** models surfaces as collections of microscopic perfect mirrors:

$$
f_r(\omega_i, \omega_o) = \frac{D(\mathbf{h}) \, G(\omega_i, \omega_o, \mathbf{h}) \, F(\omega_i, \mathbf{h})}{4 (\mathbf{n} \cdot \omega_i) (\mathbf{n} \cdot \omega_o)}
$$

where:
- $\mathbf{h} = \frac{\omega_i + \omega_o}{|\omega_i + \omega_o|}$ is the halfway vector
- $D(\mathbf{h})$ = Normal Distribution Function (NDF)
- $G(\omega_i, \omega_o, \mathbf{h})$ = Geometry function
- $F(\omega_i, \mathbf{h})$ = Fresnel term

### Normal Distribution Function (GGX/Trowbridge-Reitz)

$$
D_{GGX}(\mathbf{h}) = \frac{\alpha^2}{\pi ((\mathbf{n} \cdot \mathbf{h})^2 (\alpha^2 - 1) + 1)^2}
$$

where $\alpha = roughness^2$

### Geometry Function (Smith's method with GGX)

$$
G(\omega_i, \omega_o, \mathbf{h}) = G_1(\omega_i) \, G_1(\omega_o)
$$

$$
G_1(\omega) = \frac{2 (\mathbf{n} \cdot \omega)}{(\mathbf{n} \cdot \omega) + \sqrt{\alpha^2 + (1 - \alpha^2)(\mathbf{n} \cdot \omega)^2}}
$$

### Fresnel (Schlick Approximation)

$$
F_{Schlick}(\omega_i, \mathbf{h}) = F_0 + (1 - F_0)(1 - (\omega_i \cdot \mathbf{h}))^5
$$

where $F_0$ is the base reflectivity:
- **Dielectrics**: $F_0 \approx 0.04$ (most non-metals)
- **Metals**: $F_0 = $ base color

In [ ]:
def distribution_ggx(n_dot_h: float, roughness: float) -> float:
    """GGX/Trowbridge-Reitz normal distribution function"""
    alpha = roughness * roughness
    alpha2 = alpha * alpha
    
    n_dot_h2 = n_dot_h * n_dot_h
    
    denom = n_dot_h2 * (alpha2 - 1.0) + 1.0
    denom = math.pi * denom * denom
    
    return alpha2 / max(denom, 0.0001)

def geometry_schlick_ggx(n_dot_v: float, roughness: float) -> float:
    """Schlick-GGX geometry function (single direction)"""
    alpha = roughness * roughness
    k = alpha / 2.0  # For direct lighting; use (alpha+1)^2/8 for IBL
    
    denom = n_dot_v * (1.0 - k) + k
    return n_dot_v / max(denom, 0.0001)

def geometry_smith(n_dot_v: float, n_dot_l: float, roughness: float) -> float:
    """Smith's method combining geometry for view and light directions"""
    ggx1 = geometry_schlick_ggx(n_dot_v, roughness)
    ggx2 = geometry_schlick_ggx(n_dot_l, roughness)
    return ggx1 * ggx2

def fresnel_schlick(cos_theta: float, f0: Vec3) -> Vec3:
    """Schlick's approximation for Fresnel term"""
    return f0 + (Vec3(1, 1, 1) - f0) * math.pow(max(1.0 - cos_theta, 0.0), 5.0)

def fresnel_schlick_roughness(cos_theta: float, f0: Vec3, roughness: float) -> Vec3:
    """Fresnel-Schlick with roughness (for IBL)"""
    one_minus_roughness = Vec3(1.0 - roughness, 1.0 - roughness, 1.0 - roughness)
    max_f0_roughness = Vec3(
        max(f0.x, one_minus_roughness.x),
        max(f0.y, one_minus_roughness.y),
        max(f0.z, one_minus_roughness.z)
    )
    return f0 + (max_f0_roughness - f0) * math.pow(max(1.0 - cos_theta, 0.0), 5.0)

print("✓ PBR microfacet functions loaded")

## 3. Cook-Torrance BRDF

The complete **Cook-Torrance** specular BRDF:

$$
f_{spec} = \frac{D \, G \, F}{4 (\mathbf{n} \cdot \omega_i) (\mathbf{n} \cdot \omega_o)}
$$

### Diffuse Component (Lambertian)

$$
f_{diff} = \frac{\mathbf{c}}{\pi} (1 - m)
$$

where $m$ is metallic parameter.

### Combined BRDF

$$
f_r = f_{diff} (1 - F) + f_{spec}
$$

The Fresnel term $F$ determines energy split between diffuse and specular.

### Full PBR Shading Equation

$$
L_o = \int_{\Omega} (k_d \frac{\mathbf{c}}{\pi} + \frac{DGF}{4(\mathbf{n}\cdot\omega_i)(\mathbf{n}\cdot\omega_o)}) L_i (\mathbf{n} \cdot \omega_i) \, d\omega_i
$$

where $k_d = (1 - F)(1 - m)$ is the diffuse contribution.

In [ ]:
@dataclass
class PBRMaterial:
    """PBR material parameters"""
    albedo: Color
    metallic: float
    roughness: float
    ao: float = 1.0  # Ambient occlusion
    emission: Color = None
    
    def __post_init__(self):
        if self.emission is None:
            self.emission = Color(0, 0, 0)
    
    def get_f0(self) -> Vec3:
        """Get base reflectivity (F0)"""
        # Dielectrics have F0 ~0.04, metals use albedo
        dielectric_f0 = Vec3(0.04, 0.04, 0.04)
        albedo_vec = self.albedo.to_vec3()
        
        # Linear interpolation based on metallic
        return dielectric_f0 + (albedo_vec - dielectric_f0) * self.metallic

@dataclass
class Light:
    """Point light source"""
    position: Vec3
    color: Color
    intensity: float

def cook_torrance_brdf(normal: Vec3, view_dir: Vec3, light_dir: Vec3,
                       material: PBRMaterial) -> Color:
    """Cook-Torrance microfacet BRDF"""
    
    # Halfway vector
    halfway = (view_dir + light_dir).normalize()
    
    # Dot products
    n_dot_v = max(normal.dot(view_dir), 0.0)
    n_dot_l = max(normal.dot(light_dir), 0.0)
    n_dot_h = max(normal.dot(halfway), 0.0)
    h_dot_v = max(halfway.dot(view_dir), 0.0)
    
    # Get F0 (base reflectivity)
    f0 = material.get_f0()
    
    # Cook-Torrance terms
    D = distribution_ggx(n_dot_h, material.roughness)
    G = geometry_smith(n_dot_v, n_dot_l, material.roughness)
    F = fresnel_schlick(h_dot_v, f0)
    
    # Specular BRDF
    numerator = D * G
    denominator = 4.0 * n_dot_v * n_dot_l
    specular = numerator / max(denominator, 0.001)
    
    # Energy conservation: diffuse component
    k_d = Vec3(1, 1, 1) - F  # Fresnel determines specular, rest is diffuse
    k_d = k_d * (1.0 - material.metallic)  # Metals have no diffuse
    
    # Lambertian diffuse
    diffuse = material.albedo.to_vec3() / math.pi
    
    # Combine diffuse and specular
    brdf_diffuse = Vec3(k_d.x * diffuse.x, k_d.y * diffuse.y, k_d.z * diffuse.z)
    brdf_specular = Vec3(F.x * specular, F.y * specular, F.z * specular)
    
    brdf = brdf_diffuse + brdf_specular
    
    return Color(brdf.x, brdf.y, brdf.z)

def pbr_direct_lighting(point: Vec3, normal: Vec3, view_dir: Vec3,
                       material: PBRMaterial, lights: List[Light]) -> Color:
    """PBR direct lighting calculation"""
    
    Lo = Color(0, 0, 0)
    
    for light in lights:
        # Light direction and distance
        light_vec = light.position - point
        distance = light_vec.length()
        light_dir = light_vec.normalize()
        
        # Attenuation (inverse square law)
        attenuation = 1.0 / (distance * distance)
        radiance = light.color * (light.intensity * attenuation)
        
        # BRDF
        brdf = cook_torrance_brdf(normal, view_dir, light_dir, material)
        
        # Rendering equation
        n_dot_l = max(normal.dot(light_dir), 0.0)
        contribution = brdf * radiance * n_dot_l
        
        Lo = Lo + contribution
    
    # Ambient (very simple)
    ambient = Color(0.03, 0.03, 0.03) * material.albedo * material.ao
    
    return ambient + Lo + material.emission

print("✓ Cook-Torrance BRDF loaded")

## 4. Image-Based Lighting (IBL)

**Image-Based Lighting** uses environment maps to capture ambient lighting:

$$
L_o = \int_{\Omega} (k_d \frac{\mathbf{c}}{\pi} + f_{spec}) L_i(\omega_i) (\mathbf{n} \cdot \omega_i) \, d\omega_i
$$

### Split-Sum Approximation

The integral is split into two parts:

1. **Diffuse irradiance** (pre-convolved):
$$
L_{diff} = \mathbf{c} \cdot \text{IrradianceMap}(\mathbf{n})
$$

2. **Specular radiance** (pre-filtered + BRDF LUT):
$$
L_{spec} = \text{PrefilteredMap}(\mathbf{r}, roughness) \cdot (F \cdot \text{scale} + \text{bias})
$$

where $\mathbf{r}$ is the reflection vector.

### Environment Map Types

- **Cube map**: 6 faces (±X, ±Y, ±Z)
- **Equirectangular**: Latitude-longitude mapping
- **Spherical harmonics**: Compact representation

In [ ]:
class Ray:
    """Ray for rendering"""
    def __init__(self, origin: Vec3, direction: Vec3):
        self.origin = origin
        self.direction = direction.normalize()
    
    def at(self, t: float) -> Vec3:
        return self.origin + self.direction * t

class Sphere:
    """Sphere primitive with PBR material"""
    def __init__(self, center: Vec3, radius: float, material: PBRMaterial):
        self.center = center
        self.radius = radius
        self.material = material
    
    def intersect(self, ray: Ray) -> Optional[float]:
        """Ray-sphere intersection"""
        oc = ray.origin - self.center
        a = ray.direction.dot(ray.direction)
        half_b = oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        
        discriminant = half_b * half_b - a * c
        if discriminant < 0:
            return None
        
        sqrtd = math.sqrt(discriminant)
        t = (-half_b - sqrtd) / a
        
        if t > 0.001:
            return t
        
        t = (-half_b + sqrtd) / a
        if t > 0.001:
            return t
        
        return None
    
    def normal_at(self, point: Vec3) -> Vec3:
        return (point - self.center).normalize()

class Scene:
    """Scene with PBR materials"""
    def __init__(self):
        self.objects = []
        self.lights = []
    
    def add_object(self, obj):
        self.objects.append(obj)
    
    def add_light(self, light):
        self.lights.append(light)
    
    def intersect(self, ray: Ray) -> Optional[Tuple[float, Sphere]]:
        closest_t = float('inf')
        hit_object = None
        
        for obj in self.objects:
            t = obj.intersect(ray)
            if t is not None and t < closest_t:
                closest_t = t
                hit_object = obj
        
        if hit_object:
            return (closest_t, hit_object)
        return None

print("✓ Scene classes loaded")

In [ ]:
def render_pbr(scene: Scene, camera_pos: Vec3, camera_dir: Vec3, 
               width: int, height: int) -> np.ndarray:
    """Simple PBR renderer"""
    image = np.zeros((height, width, 3))
    
    # Camera setup (simple orthographic-like)
    up = Vec3(0, 1, 0)
    right = camera_dir.cross(up).normalize()
    true_up = right.cross(camera_dir)
    
    aspect = width / height
    fov_scale = 2.0
    
    for j in range(height):
        if j % 20 == 0:
            print(f"Scanline {j}/{height}")
        
        for i in range(width):
            # Normalized device coordinates
            u = (2.0 * i / width - 1.0) * aspect * fov_scale
            v = (1.0 - 2.0 * j / height) * fov_scale
            
            # Ray direction
            ray_dir = (camera_dir + right * u + true_up * v).normalize()
            ray = Ray(camera_pos, ray_dir)
            
            # Intersect scene
            hit = scene.intersect(ray)
            
            if hit:
                t, obj = hit
                hit_point = ray.at(t)
                normal = obj.normal_at(hit_point)
                view_dir = -ray_dir
                
                # PBR shading
                color = pbr_direct_lighting(hit_point, normal, view_dir,
                                           obj.material, scene.lights)
                
                # Gamma correction
                color = Color(
                    math.pow(max(0, color.r), 1.0/2.2),
                    math.pow(max(0, color.g), 1.0/2.2),
                    math.pow(max(0, color.b), 1.0/2.2)
                )
                
                image[j, i] = color.clamp(0, 1).to_tuple()
            else:
                # Background
                image[j, i] = (0.1, 0.1, 0.15)
    
    return image

print("✓ PBR renderer loaded")

## Example 1: Material Comparison (Varying Roughness)

In [ ]:
# Create scene with spheres of varying roughness
scene = Scene()

# Materials with different roughness values
roughness_values = [0.0, 0.25, 0.5, 0.75, 1.0]
positions = [-4, -2, 0, 2, 4]

for roughness, x_pos in zip(roughness_values, positions):
    mat = PBRMaterial(
        albedo=Color(1.0, 0.8, 0.1),  # Gold color
        metallic=1.0,
        roughness=roughness,
        ao=1.0
    )
    scene.add_object(Sphere(Vec3(x_pos, 0, 0), 0.8, mat))

# Add lights
scene.add_light(Light(Vec3(5, 5, 5), Color(1, 1, 1), 100))
scene.add_light(Light(Vec3(-5, 5, 5), Color(1, 1, 1), 80))
scene.add_light(Light(Vec3(0, -3, 5), Color(0.5, 0.5, 0.8), 50))

# Render
print("Rendering roughness comparison...")
camera_pos = Vec3(0, 0, 8)
camera_dir = Vec3(0, 0, -1)
image = render_pbr(scene, camera_pos, camera_dir, 500, 200)

plt.figure(figsize=(14, 6))
plt.imshow(image)
plt.title('PBR Metallic Spheres: Roughness 0.0, 0.25, 0.5, 0.75, 1.0 (left to right)')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice: Left = mirror-like, Right = rough/diffuse")

## Example 2: Metallic vs Dielectric

In [ ]:
# Scene comparing metallic and dielectric materials
scene2 = Scene()

# Row 1: Dielectrics (metallic = 0)
for i, roughness in enumerate([0.0, 0.3, 0.6, 0.9]):
    mat = PBRMaterial(
        albedo=Color(0.8, 0.2, 0.2),  # Red
        metallic=0.0,
        roughness=roughness
    )
    scene2.add_object(Sphere(Vec3(-3 + i*2, 1.5, 0), 0.6, mat))

# Row 2: Metals (metallic = 1)
for i, roughness in enumerate([0.0, 0.3, 0.6, 0.9]):
    mat = PBRMaterial(
        albedo=Color(0.8, 0.2, 0.2),  # Red
        metallic=1.0,
        roughness=roughness
    )
    scene2.add_object(Sphere(Vec3(-3 + i*2, -1.5, 0), 0.6, mat))

# Lights
scene2.add_light(Light(Vec3(5, 5, 5), Color(1, 1, 1), 100))
scene2.add_light(Light(Vec3(-5, 5, 5), Color(1, 1, 1), 100))

# Render
print("Rendering metallic vs dielectric...")
image2 = render_pbr(scene2, Vec3(0, 0, 10), Vec3(0, 0, -1), 500, 300)

plt.figure(figsize=(14, 8))
plt.imshow(image2)
plt.title('Top row: Dielectric (metallic=0), Bottom row: Metal (metallic=1)')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice: Metals reflect environment color, dielectrics show base color.")

## Example 3: Real-World Material Library

In [ ]:
# Common real-world PBR materials
materials_library = {
    'Gold': PBRMaterial(Color(1.0, 0.765, 0.336), metallic=1.0, roughness=0.2),
    'Silver': PBRMaterial(Color(0.972, 0.960, 0.915), metallic=1.0, roughness=0.1),
    'Copper': PBRMaterial(Color(0.955, 0.637, 0.538), metallic=1.0, roughness=0.3),
    'Iron': PBRMaterial(Color(0.560, 0.570, 0.580), metallic=1.0, roughness=0.5),
    'Plastic': PBRMaterial(Color(0.8, 0.1, 0.1), metallic=0.0, roughness=0.4),
    'Rubber': PBRMaterial(Color(0.1, 0.1, 0.1), metallic=0.0, roughness=0.9),
}

scene3 = Scene()

# Add spheres with different materials
material_list = list(materials_library.items())
positions_3d = [
    Vec3(-4, 1, 0),
    Vec3(-1.5, 1, 0),
    Vec3(1, 1, 0),
    Vec3(-4, -1.5, 0),
    Vec3(-1.5, -1.5, 0),
    Vec3(1, -1.5, 0),
]

for (name, material), pos in zip(material_list, positions_3d):
    scene3.add_object(Sphere(pos, 0.7, material))

# Lights
scene3.add_light(Light(Vec3(6, 6, 6), Color(1, 1, 1), 120))
scene3.add_light(Light(Vec3(-6, 6, 6), Color(0.9, 0.9, 1.0), 100))
scene3.add_light(Light(Vec3(0, -5, 8), Color(1.0, 0.9, 0.8), 80))

# Render
print("Rendering material library...")
image3 = render_pbr(scene3, Vec3(0, 0, 10), Vec3(0, 0, -1), 500, 350)

plt.figure(figsize=(14, 10))
plt.imshow(image3)
plt.title('PBR Material Library: Gold, Silver, Copper (top), Iron, Plastic, Rubber (bottom)')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Notice different materials respond differently to same lighting.")

## Summary

**Physically Based Rendering (PBR)** provides a standardized, physically accurate shading model:

### Key Principles

1. **Energy Conservation**: Reflected light ≤ incident light
2. **Physical Correctness**: Based on real-world physics and measurements
3. **Consistency**: Materials look correct under all lighting conditions
4. **Artist-Friendly**: Intuitive parameters (albedo, metallic, roughness)

### Core Components

1. **Microfacet BRDF**: Models surfaces as microscopic mirrors
   - Normal Distribution Function (GGX)
   - Geometry Function (Smith)
   - Fresnel Term (Schlick)

2. **Material Parameters**:
   - Base Color (Albedo)
   - Metallic (0 = dielectric, 1 = metal)
   - Roughness (0 = smooth, 1 = rough)
   - Normal maps
   - Ambient Occlusion

3. **Lighting**:
   - Direct lighting (analytical lights)
   - Image-based lighting (environment maps)
   - Global illumination (path tracing)

### Material Types

- **Dielectrics** (metallic = 0): Plastic, wood, stone, skin
  - F0 ≈ 0.04 (4% reflectance)
  - Colored diffuse, white specular

- **Metals** (metallic = 1): Gold, silver, iron
  - F0 = albedo color
  - No diffuse, colored specular

### Applications

PBR is the industry standard in:
- Game engines (Unreal, Unity)
- Film rendering (Disney, Pixar)
- Product visualization
- Virtual reality

### Advantages

- Physically plausible results
- Reusable materials across different lighting
- Predictable behavior
- Efficient parameter space
- Real-time capable with optimizations